In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [25]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
# Preprcessing The Data
Label_Encoder_Gender=LabelEncoder()
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [27]:
### Encoding the categorical data
Label_Encoder_Gender=LabelEncoder()
data['Gender']=Label_Encoder_Gender.fit_transform(data['Gender'])

In [28]:
## One Hot Encoding "GEoGraphy"
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [29]:
# Combine the encoded columns with the original dataframe
data = pd.concat([data.drop(['Geography'], axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [30]:
# Split the data into features and target variable
X=data.drop('EstimatedSalary',axis=1) 
y=data['EstimatedSalary'] 


In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [32]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [34]:
## save the econded data and sclaer for later use
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(Label_Encoder_Gender, f)

with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [35]:
## ANN With REgression Statementd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [36]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

## Complie The data
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

model.summary()

/home/gaurav/Projects/ANN(Classifiaction)/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-15 17:43:42.687083: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


## Setup The TensorBoard
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
TensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [38]:
# Set Up The erarly Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True) 

In [39]:
history=model.fit(X_train,y_train,validation_split=0.2,epochs=100,batch_size=32,validation_data=(X_test,y_test),callbacks=[TensorBoard_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 13435972608.0000 - mae: 100552.4219 - val_loss: 13012841472.0000 - val_mae: 98547.8281
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13353030656.0000 - mae: 100225.5469 - val_loss: 12713630720.0000 - val_mae: 97035.1484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 12983555072.0000 - mae: 98548.6641 - val_loss: 11909463040.0000 - val_mae: 92965.3984
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11928439808.0000 - mae: 93147.9844 - val_loss: 10545912832.0000 - val_mae: 86078.5156
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 10432448512.0000 - mae: 85734.3750 - val_loss: 8779260928.0000 - val_mae: 77124.9219
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8755737600.0000 - mae: 77509.2578 - val_loss: 6931707904.0000 - val_mae: 67882.7500
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 6770734592.0000 - mae: 67273.8516 - val_loss: 5355206656.0

In [40]:
## Evaluate The Model
%load_ext tensorboard


In [43]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 29700), started 0:00:13 ago. (Use '!kill 29700' to kill it.)

In [44]:
## Evalute Model in test Data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f"Test mae: {test_mae}")    

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3417178880.0000 - mae: 50820.3906
Test mae: 50068.12109375


In [45]:
model.save('regression_model.h5')